In [ ]:
import geatpy as ea
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU,Dropout
from pyDOE import lhs
from keras import optimizers
from numpy import concatenate
import json
import numpy as np
import geatpy as ea
import pandas as pd
import matplotlib.pyplot as plt  # 用来绘图的库
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import AdaBoostRegressor
from numpy import concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras import optimizers
from sklearn.svm import SVR
from matplotlib import pyplot
import random

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from numpy import concatenate
import pandas as pd
import matplotlib.pyplot as plt  # 用来绘图的库
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.ensemble import AdaBoostRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras import optimizers
from sklearn.svm import SVR
import numpy as np
import re
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import tensorflow as tf
from tensorflow.compat.v1 import InteractiveSession
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import random
EA_NAME = ['soea_DE_best_1_bin_templet','soea_DE_best_1_L_templet','soea_DE_rand_1_bin_templet',
           'soea_SGA_templet','soea_EGA_templet','soea_steadyGA_templet','soea_SEGA_templet','soea_SEGA_templet']

In [ ]:
def calcul_indicator(y,yhat):
    EV = 1 - np.var(y - yhat)/np.var(y)
    MAE = np.sum(np.abs((y - yhat)))/len(y)
    MSE = np.sum(np.multiply((y - yhat),(y - yhat)))/len(y)
    RMSE = sqrt(MSE)
    R_square = 1 - (((y - yhat)**2).sum())/(((y - y.mean())**2).sum())
    return [EV,MAE,MSE,RMSE,R_square]

def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    
def inverse_transform(scaler, data):
    return scaler.inverse_transform(data)
#原始克隆选择算法
class ModifiedClonalSelectionAlgorithm:
    def __init__(self, fitness_func, num_parameters, population_size, clone_factor, mutation_rate, num_generations, early_stop):
        self.fitness_func = fitness_func
        self.num_parameters = num_parameters
        self.population_size = population_size
        self.clone_factor = clone_factor
        self.mutation_rate = mutation_rate
        self.num_generations = num_generations
        self.early_stop = early_stop  # 提前停止的代数阈值
        self.rmse_history = []

    def initialize_population(self):
        # 使用均匀随机初始化
        initial_population = np.random.rand(self.population_size, self.num_parameters)
        return initial_population

    def clone(self, population, fitness_values):
        # 计算最大适应度值
        max_fitness = np.max(fitness_values)
        min_fitness = np.min(fitness_values)
        # 计算个体的调整适应度值,确保值在(0, 1]范围内
        adjusted_fitness = (max_fitness - fitness_values + 1e-10) / (max_fitness - min_fitness + 1e-10)

        clones = []
        for antibody, fitness in zip(population, adjusted_fitness):
            # 计算克隆数量,根据调整适应度值和克隆因子
            num_clones = int(self.clone_factor * fitness)
            clones.extend([antibody.copy() for _ in range(num_clones)])

        # 如果克隆数量不足以填充种群,随机生成个体进行补充
        if len(clones) < self.population_size:
            num_new = self.population_size - len(clones)
            new_antibodies = np.random.rand(num_new, self.num_parameters)
            clones.extend(new_antibodies)

        return np.array(clones)

    def mutate(self, clones):
        # 使用固定的变异率
        mutation_mask = np.random.rand(len(clones), self.num_parameters) < self.mutation_rate
        gaussian_noise = np.random.normal(0, self.mutation_rate, size=np.sum(mutation_mask))
        clones[mutation_mask] = np.clip(clones[mutation_mask] + gaussian_noise, 0, 1)  # 将变异后的参数值裁剪到 [0, 1] 区间内
        return clones

    def select(self, population, clones):
        combined = np.vstack((population, clones))
        fitness_values = np.apply_along_axis(self.fitness_func, 1, combined)
        best_indices = np.argsort(fitness_values)[:self.population_size]
        return combined[best_indices]

    def optimize(self):
        population = self.initialize_population()
        best_fitness = np.inf  # 初始化为无穷大
        no_improvement_count = 0  # 记录连续无改进的代数
        for _ in range(self.num_generations):
            fitness_values = np.apply_along_axis(self.fitness_func, 1, population)
            curr_best_fitness = np.min(fitness_values)

            # 如果当前最佳适应度值比之前的最佳值更好,则更新最佳值,并重置无改进计数器
            if curr_best_fitness < best_fitness:
                best_fitness = curr_best_fitness
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            # 如果连续多代无改进,则提前停止
            if no_improvement_count >= self.early_stop:
                break

            clones = self.clone(population, fitness_values)
            mutated_clones = self.mutate(clones)
            population = self.select(population, mutated_clones)

        best_index = np.argmin(np.apply_along_axis(self.fitness_func, 1, population))
        return population[best_index] 
#优化的GRU模型(原始克隆选择算法调优)
def MOptimizedGRU(train_X, train_y, test_X, test_y, calcul_indicator):
    def GRU_fitness(params):
        set_random_seed(42)
        train_X_GRU = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X_GRU = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

        num_units = 50 + int(params[0] * 150)  # GRU 单元数 (50 to 150)
        dropout_rate = params[2]*0.5  # Dropout率 (0 to 0.5)
        learning_rate = params[1] * 0.01 + 0.0001  # 学习率 (0.0001 to 0.01)  
        epochs = 30 + int(params[3] * 100)  # epochs (50 to 150)
        batch_size = 8+ int(params[4] * 120)  # batch_size (2 to 100)
        
        model = Sequential()
        model.add(GRU(num_units, input_shape=(train_X_GRU.shape[1], train_X_GRU.shape[2])))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1))

   
        decay_rate = 0.000003
        momentum = 0.5
        sgd = optimizers.gradient_descent_v2.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
        model.compile(loss='mse', optimizer=sgd)

        history = model.fit(train_X_GRU, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_X_GRU, test_y), verbose=1, shuffle=False)

        yhat = model.predict(test_X_GRU)
        yhat = yhat.reshape(len(yhat), 1)
        test_y_reshaped = test_y.reshape(len(test_y), 1)

        indicators = calcul_indicator(test_y_reshaped, yhat)
        fitness = indicators[3] # RMSE作为适应度值
        return fitness


    # 使用原始的克隆选择算法找到最优参数
    ain =  ModifiedClonalSelectionAlgorithm(GRU_fitness,5 , 20, 4, 0.5, 20, 4)   #克隆选择参数设置
    best_params = ain.optimize()
    plt.plot(ain.rmse_history, label='RMSE')
    plt.xlabel('Generation')
    plt.ylabel('RMSE')
    plt.title('RMSE over Generations')
    plt.legend()
    plt.show()
    set_random_seed(42)
    # 使用最优参数训练GRU模型
    best_num_units = 50 + int(best_params[0] * 150)
    best_learning_rate = best_params[1] * 0.01 + 0.0001
    best_dropout = best_params[2]*0.5
    best_epochs = 30 + int(best_params[3] * 100)  # epochs (50 to 150)
    best_batch_size = 8+ int(best_params[4] * 120)
    
    train_X_GRU = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X_GRU = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(GRU(best_num_units, input_shape=(train_X_GRU.shape[1], train_X_GRU.shape[2])))
    model.add(Dropout(best_dropout))
    model.add(Dense(1))

    decay_rate = 0.00003
    momentum = 0.5
    sgd = optimizers.gradient_descent_v2.SGD(learning_rate=best_learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss='mse', optimizer=sgd)

    history = model.fit(train_X_GRU, train_y, epochs=best_epochs , batch_size= best_batch_size, validation_data=(test_X_GRU, test_y), verbose=1, shuffle=False)

    yhat = model.predict(test_X_GRU)
    yhat = yhat.reshape(len(yhat), 1)
    test_y_reshaped = test_y.reshape(len(test_y), 1)

    #绘制损失曲线
    plt.cla()
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.ylabel('loss/val_loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()
    
    #返回模型评估结果和最优参数
    return calcul_indicator(test_y_reshaped, yhat),yhat
class AClonalSelectionAlgorithm:
    def __init__(self, fitness_func, num_parameters, population_size, clone_factor, max_mutation_rate, min_mutation_rate, num_generations,early_stop):
        self.fitness_func = fitness_func
        self.num_parameters = num_parameters
        self.population_size = population_size
        self.clone_factor = clone_factor
        self.max_mutation_rate = max_mutation_rate
        self.min_mutation_rate = min_mutation_rate
        self.num_generations = num_generations
        self.early_stop = early_stop  # 提前停止的代数阈值
        self.rmse_history = []

    def initialize_population(self):
        # 使用拉丁超立方抽样生成初始种群
        initial_population = lhs(self.num_parameters, samples=self.population_size)
        return initial_population
    def create_antibody(self):
        # 生成一个新的抗体（候选解），每个参数值在 [0, 1] 范围内
        return np.random.rand(self.num_parameters)
    def clone(self, population, fitness_values):
        # 计算最大适应度值
        max_fitness = np.max(fitness_values)
        min_fitness = np.min(fitness_values)
        # 计算个体的调整适应度值,确保值在(0, 1]范围内
        adjusted_fitness = (max_fitness - fitness_values + 1e-10) / (max_fitness - min_fitness + 1e-10)
        
        # 创建一个字典来存储个体及其对应的调整适应度值
        fitness_dict = {tuple(antibody): fitness for antibody, fitness in zip(population, adjusted_fitness)}
        
        clones = []
        for antibody, fitness in zip(population, adjusted_fitness):
            # 计算克隆数量,根据调整适应度值和克隆因子
            num_clones = int(self.clone_factor * fitness)
            
            # 生成克隆个体
            clones.extend([antibody.copy() for _ in range(num_clones)])
        
        # 如果克隆数量不足以填充种群,随机生成个体进行补充
        if len(clones) < self.population_size:
            num_new = self.population_size - len(clones)
            new_antibodies = [self.create_antibody() for _ in range(num_new)]
            clones.extend(new_antibodies)
        # 如果克隆数量超过种群大小,删除适应度差的多余的个体
        elif len(clones) > self.population_size:
             clones = sorted(clones, key=lambda x: fitness_dict.get(tuple(x), -np.inf), reverse=True)[:self.population_size]
        
        return np.array(clones)
    
    def adaptive_mutation(self, fitness, min_fitness, max_fitness):
        relative_fitness = (fitness - min_fitness) / (max_fitness - min_fitness + 1e-10)  # 添加一个小的正数,以避免除以零
        mutation_rate = self.max_mutation_rate - (self.max_mutation_rate - self.min_mutation_rate) * relative_fitness
        return np.clip(mutation_rate, self.min_mutation_rate, self.max_mutation_rate)
        
  
    def mutate(self, clones, fitness_values):
        min_fitness = np.min(fitness_values)
        max_fitness = np.max(fitness_values)
        for idx, clone in enumerate(clones):
            mutation_rate = self.adaptive_mutation(fitness_values[idx], min_fitness, max_fitness)
            mutation_mask = np.random.rand(self.num_parameters) < mutation_rate
            gaussian_noise = np.random.normal(0, mutation_rate, size=np.sum(mutation_mask))
            clone[mutation_mask] = np.clip(clone[mutation_mask] + gaussian_noise, 0, 1)  # 将变异后的参数值裁剪到 [0, 1] 区间内
        return clones
        

    def select(self, population, clones):
        combined = np.vstack((population, clones))
        fitness_values = np.apply_along_axis(self.fitness_func, 1, combined)
        best_indices = np.argsort(fitness_values)[:self.population_size]
        return combined[best_indices]
    
    def optimize(self):
        population = self.initialize_population()
        best_fitness = np.inf  # 初始化为无穷大
        no_improvement_count = 0  # 记录连续无改进的代数
        for _ in range(self.num_generations):
            fitness_values = np.apply_along_axis(self.fitness_func, 1, population)
            curr_best_fitness = np.min(fitness_values)

             # 将当前代的最优适应度值保存到历史记录中
            self.rmse_history.append(curr_best_fitness)
            
            # 如果当前最佳适应度值比之前的最佳值更好,则更新最佳值,并重置无改进计数器
            if curr_best_fitness < best_fitness:
                best_fitness = curr_best_fitness
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            # 如果连续多代无改进,则提前停止-早停
            if no_improvement_count >= self.early_stop:
                break
            clones = self.clone(population, fitness_values)
            mutated_clones = self.mutate(clones, fitness_values)
            population = self.select(population, mutated_clones)
        best_index = np.argmin(np.apply_along_axis(self.fitness_func, 1, population))
        return population[best_index] 
    


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def RandomForest(train_X,train_y,test_X,test_y):
    model = RandomForestRegressor(n_estimators=25)
    history = model.fit(train_X, train_y)
    yhat = (history.predict(test_X)).T
    yhat = yhat.reshape(len(yhat), 1)
    test_y = test_y.reshape(len(test_y), 1)
    return calcul_indicator(test_y, yhat),yhat

def Adaboost(train_X,train_y,test_X,test_y):
    model = AdaBoostRegressor(n_estimators=50)
    history = model.fit(train_X, train_y)
    yhat = (history.predict(test_X)).T
    yhat = yhat.reshape(len(yhat), 1)
    test_y = test_y.reshape(len(test_y), 1)
    return calcul_indicator(test_y, yhat),yhat

def LR(train_X,train_y,test_X,test_y):
    model = Sequential()
    model.add(Dense(1, input_dim=train_X.shape[1]))
    model.compile(loss='mse', optimizer='Adam')
    history = model.fit(train_X, train_y, batch_size=60, epochs=32, validation_data=(test_X, test_y), verbose=1,
                        shuffle=False)
    yhat = model.predict(test_X)
    yhat = yhat.reshape(len(yhat), 1)
    test_y = test_y.reshape(len(test_y), 1)
    return calcul_indicator(test_y, yhat),yhat

def SVRR(train_X,train_y,test_X,test_y):
    model = SVR(kernel='poly')
    history = model.fit(train_X, train_y)
    yhat = (history.predict(test_X)).T
    yhat = yhat.reshape(len(yhat), 1)
    test_y = test_y.reshape(len(test_y), 1)
    return calcul_indicator(test_y, yhat),yhat


def GRUM(train_X,train_y,test_X,test_y):
    # 将数据转换成3D进行输入
    set_random_seed(42)
    train_X_GRU = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X_GRU = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    model = Sequential()
    model.add(GRU(100, input_shape=(train_X_GRU.shape[1], train_X_GRU.shape[2])))  #神经元数可以调少一点，比如80
    model.add(Dropout(0.2))
    model.add(Dense(1))
    # Compile model
    learning_rate = 0.0003   #学习率可以试一试别的，一般在0.001到0.0001之间，可以试试0.0003
    decay_rate = 0.000003    #可以调
    momentum = 0.5
    sgd = optimizers.gradient_descent_v2.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate,
                                             nesterov=False)
    model.compile(loss='mse', optimizer=sgd)
    # fit network
    history = model.fit(train_X_GRU, train_y, epochs=100, batch_size=32, validation_data=(test_X_GRU, test_y),
                        verbose=1, shuffle=False)
    yhat = model.predict(test_X_GRU)
    yhat = yhat.reshape(len(yhat), 1)
    plt.cla()
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.ylabel('loss/val_loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()
    test_y = test_y.reshape(len(test_y), 1)
    return calcul_indicator(test_y,yhat),yhat



#优化的GRU模型
def OptimizedGRU(train_X, train_y, test_X, test_y, calcul_indicator):
    def GRU_fitness(params):
        set_random_seed(42)
        train_X_GRU = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X_GRU = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

        num_units = 50 + int(params[0] * 150)  # GRU 单元数 (50 to 150)
        dropout_rate = params[2]*0.5  # Dropout率 (0 to 0.5)
        learning_rate = params[1] * 0.01 + 0.0001  # 学习率 (0.0001 to 0.01)  
        epochs = 30 + int(params[3] * 100)  # epochs (50 to 150)
        batch_size = 8+ int(params[4] * 120)  # batch_size (2 to 100)
        
        model = Sequential()
        model.add(GRU(num_units, input_shape=(train_X_GRU.shape[1], train_X_GRU.shape[2])))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1))

   
        decay_rate = 0.000003
        momentum = 0.5
        sgd = optimizers.gradient_descent_v2.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
        model.compile(loss='mse', optimizer=sgd)

        history = model.fit(train_X_GRU, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_X_GRU, test_y), verbose=1, shuffle=False)

        yhat = model.predict(test_X_GRU)
        yhat = yhat.reshape(len(yhat), 1)
        test_y_reshaped = test_y.reshape(len(test_y), 1)

        indicators = calcul_indicator(test_y_reshaped, yhat)
        fitness = indicators[3] # RMSE作为适应度值
        return fitness


    # 使用免疫网络算法找到最优参数
    ain = AClonalSelectionAlgorithm(GRU_fitness,5 , 20, 4, 0.5, 0.05, 20, 4)   #克隆选择参数设置
    best_params = ain.optimize()
    plt.plot(ain.rmse_history, label='RMSE')
    plt.xlabel('Generation')
    plt.ylabel('RMSE')
    plt.title('RMSE over Generations')
    plt.legend()
    plt.show()
    set_random_seed(42)
    # 使用最优参数训练GRU模型
    best_num_units = 50 + int(best_params[0] * 150)
    best_learning_rate = best_params[1] * 0.01 + 0.0001
    best_dropout = best_params[2]*0.5
    best_epochs = 30 + int(best_params[3] * 100)  # epochs (50 to 150)
    best_batch_size = 8+ int(best_params[4] * 120)
    
    train_X_GRU = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X_GRU = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(GRU(best_num_units, input_shape=(train_X_GRU.shape[1], train_X_GRU.shape[2])))
    model.add(Dropout(best_dropout))
    model.add(Dense(1))

    decay_rate = 0.00003
    momentum = 0.5
    sgd = optimizers.gradient_descent_v2.SGD(learning_rate=best_learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss='mse', optimizer=sgd)

    history = model.fit(train_X_GRU, train_y, epochs=best_epochs , batch_size= best_batch_size, validation_data=(test_X_GRU, test_y), verbose=1, shuffle=False)

    yhat = model.predict(test_X_GRU)
    yhat = yhat.reshape(len(yhat), 1)
    test_y_reshaped = test_y.reshape(len(test_y), 1)

    # 绘制损失曲线
    plt.cla()
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.ylabel('loss/val_loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

    # 返回模型评估结果和最优参数
    return calcul_indicator(test_y_reshaped, yhat),yhat

In [ ]:
if __name__ == '__main__':
    areas = ['DS1']
    indicator = ['EV','MAE','MSE','RMSE','R_square']

    # 如果是需要改动对比的算法，在这里进行改进即可
    classifers = ['RF','Adaboost','LinearRegression','SVR','GRU','GGAP-GRU','AClonalSelection','GGAP-MClonalSelection','GGAP-AClonalSelection']
    # classifers = ['RF','Adaboost','LinearRegression','SVR','GRU','SEGA-GRU']
    df = pd.DataFrame(columns=classifers, index=indicator)
    #df = pd.DataFrame()
    for area in areas:
        path = './{}/'.format(area)
        pre_data_df = pd.read_csv(path+'8area_{}.csv'.format(area))
        
        values = pre_data_df.values
        values = values.astype('float32')
        
        # 确保震级在第一列，并分离特征和目标
        target = values[:, 0].reshape(-1, 1)
        features = values[:, 1:]
        
        # 分别对特征和目标进行标准化
        feature_scaler = MinMaxScaler(feature_range=(0, 1))
        target_scaler = MinMaxScaler(feature_range=(0, 1))
        
        scaled_features = feature_scaler.fit_transform(features)
        scaled_target = target_scaler.fit_transform(target)
        
        # 将标准化后的特征和目标合并
        scaled_data = np.hstack((scaled_target, scaled_features))
        
        # 将数据转换为监督学习格式
        reframed = series_to_supervised(scaled_data, 1, 1)
        reframed.drop(reframed.columns[[i for i in range(int(reframed.shape[1] / 2) + 1, reframed.shape[1])]], axis=1, inplace=True)
        
        values = reframed.values
        n_train_hours = int(reframed.shape[0] * 0.7)
        train = values[:n_train_hours]
        test = values[n_train_hours:]
        train_X, train_y = train[:, 1:], train[:, 0]  # 训练集特征和目标
        test_X, test_y = test[:, 1:], test[:, 0]  # 测试集特征和目标
        
        # 对测试集的真实值进行逆标准化
        test_y_inverse = target_scaler.inverse_transform(test_y.reshape(-1, 1)).flatten()
        predictions_df = pd.DataFrame({'True Value': test_y_inverse})
        # 对每种方法进行遍历
        for method in classifers:
            data_df  =pre_data_df
            if method[:4]=='GGAP':
                res = pd.read_csv(path+'soea_GGAP_SGA_templet/optPop/Chrom.csv',header=None)
                values = res.values
                features = data_df.columns[1:]
                best_features = []
                best_features.append('label_M')
                for index in range(values.shape[1]):
                     if values[0][index] == 1:#筛选出来的特征，如果是1则选上，如果是0，则不选。
                         best_features.append(features[index])
                data_df = data_df[best_features]
            values = data_df.values
            values = values.astype('float32')
            # 确保震级在第一列，并分离特征和目标
            target = values[:, 0].reshape(-1, 1)
            features = values[:, 1:]
            
            # 分别对特征和目标进行标准化
            scaled_features = feature_scaler.fit_transform(features)
            scaled_target = target_scaler.fit_transform(target)
            
            # 将标准化后的特征和目标合并
            scaled_data = np.hstack((scaled_target, scaled_features))
            
            # 将数据转换为监督学习格式
            reframed = series_to_supervised(scaled_data, 1, 1)
            reframed.drop(reframed.columns[[i for i in range(int(reframed.shape[1] / 2) + 1, reframed.shape[1])]], axis=1, inplace=True)
            
            values = reframed.values
            n_train_hours = int(reframed.shape[0] * 0.7)
            train = values[:n_train_hours]
            test = values[n_train_hours:]
            train_X, train_y = train[:, 1:], train[:, 0]  # 训练集特征和目标
            test_X, test_y = test[:, 1:], test[:, 0]  # 测试集特征和目标
            if re.search('RF', method):
                # df.loc[:,method] = RandomForest(train_X,train_y,test_X,test_y)
                indicators, yhat = RandomForest(train_X, train_y, test_X, test_y)
            elif re.search('Adaboost',method):
                # df.loc[:,method] = Adaboost(train_X,train_y,test_X,test_y)
                indicators, yhat = Adaboost(train_X, train_y, test_X, test_y)
            elif re.search('LinearRegression',method):
                # df.loc[:,method] = LR(train_X,train_y,test_X,test_y)
                indicators, yhat = LR(train_X, train_y, test_X, test_y)
            elif re.search('SVR', method):
                # df.loc[:,method] = SVRR(train_X,train_y,test_X,test_y)
                indicators, yhat = SVRR(train_X, train_y, test_X, test_y)
            elif re.search('GRU', method):
                # df.loc[:,method] = GRUM(train_X,train_y,test_X,test_y)
                indicators, yhat = GRUM(train_X, train_y, test_X, test_y)
            elif re.search('MClonalSelection', method):
                indicators,yhat = MOptimizedGRU(train_X,train_y,test_X,test_y, calcul_indicator)
            #     df.loc[:,method]=indicators
            elif re.search('AClonalSelection', method):
                indicators,yhat = OptimizedGRU(train_X, train_y, test_X, test_y, calcul_indicator)
            #     df.loc[:,method]=indicators
            df.loc[:, method] = indicators
            
            # 对预测值进行逆标准化
            yhat_inverse = target_scaler.inverse_transform(yhat).flatten()
            predictions_df[method] = yhat_inverse
        df.to_csv(path+'comparisioncsa_methods_6_26{}.csv'.format(area))
        predictions_df.to_csv(path + 'earthgrupredict.csv', index=False)